# Continuous Optimisation HW1

In [ ]:
# Imports
import numpy as np
import scipy
import matplotlib as plt

# Define dimensions
K = 1
n = 2

# Helper functions
def plot_surface(f: function):
    from matplotlib import cm
    from matplotlib.ticker import LinearLocator

    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

    # Make data.
    X = np.arange(-5, 5, 0.25)
    Y = np.arange(-5, 5, 0.25)
    X, Y = np.meshgrid(X, Y)
    R = np.sqrt(X**2 + Y**2)
    Z = np.sin(R)

    # Plot the surface.
    surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                        linewidth=0, antialiased=False)

    # Customize the z axis.
    ax.set_zlim(-1.01, 1.01)
    ax.zaxis.set_major_locator(LinearLocator(10))
    # A StrMethodFormatter is used automatically
    ax.zaxis.set_major_formatter('{x:.02f}')

    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)

    plt.show()

# Question 1
Implement phi(x, P), bigphi(X, P), f(X, P, y).

In [64]:
# Question 1
#TODO Vectorise all the functions


def h(x: np.ndarray) -> float:
    """
    Gaussian filter

    :x: np.ndarray[1x2]
    """
    return np.e**(-np.inner(x, x)/0.01)  # Always take sigma = 0.1


def phi(x, P):
    """
    Calculate contribution of each 'true' star to observed image
    
    :x: np.ndarray[1 x 2]
    :P: np.ndarray[2 x n^2]
    """
    global K, n
    _phi = np.zeros((n**2, 1))
    for i in range(n**2):
        _phi[i] = (h(P[:, i] - x))
    print("phi function complete!")
    return _phi


def bigphi(X, P):
    """
    Calculate image observed, based on K-star positions X
    
    :X: np.ndarray[2 x K]
    :P: np.ndarray[2 x n^2]
    """
    global K, n
    bigphi = np.zeros((n**2, 1))
    for i in range(K):
        bigphi += phi(X[:, i], P)
    
    print("bigphi function complete!")
    return bigphi


def  f(X, P, y):
    """
    Calculate squared error of estimate bigphi(X)
    
    :X: np.ndarray[2 x K]
    :P: np.ndarray[2 x n^2]
    :y: np.ndarray[1 x n^2]
    """
    global K, n
    print(bigphi(X, P))
    return (1/(2*n**2)) * np.linalg.norm(bigphi(X, P)-y)**2




# Question 2

In [75]:
true_positions = np.array([[0], [0]])
positions = np.array([[0.2, 0.2], [-0.2, 0.2], [-0.2, -0.2], [0.2, -0.2]]).T
y = np.array([[0.2], [0.2], [0.2], [0.2]])
print(f(np.array([[0.2], [0.2]]), positions, y))

phi function complete!
bigphi function complete!
[[1.00000000e+00]
 [1.12535175e-07]
 [1.26641655e-14]
 [1.12535175e-07]]
phi function complete!
bigphi function complete!
0.09499998874648508


# Question 3